In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
YEARS = range(1990, 2019) #2019)

In [3]:
dfs = []
for i in YEARS:
    df_ = pd.read_csv('origin/IRS_Migration_Matrix_%d.csv' % i,
                      header=0, 
                      dtype={'Unnamed: 0': str})
    df_.index = df_['Unnamed: 0']
    df_.rename(columns={'Unnamed: 0': 'fipscounty'}, inplace=True)
    dfs.append(df_)
    

In [41]:
meta_df = pd.read_csv('origin/CountyName_MSAName.csv', 
                      dtype={'fipscounty': str, 'msa':str} )
meta_df = meta_df.drop_duplicates()
meta_df = meta_df.dropna(subset=['msa'])

In [42]:
meta_df.loc[:, 'msa'].map(lambda x: len(x))>2

0        True
1        True
2       False
3       False
4        True
        ...  
3288    False
3289     True
3290     True
3291     True
3292    False
Name: msa, Length: 3288, dtype: bool

In [44]:
sum(meta_df.loc[:, 'msa'] == '0040')

1

In [31]:
meta_df = meta_df.loc[meta_df.loc[:, 'msa'].map(lambda x: len(x))>2, :]

In [32]:
# df_t = meta_df[['fipscounty', 'msa', 'msaname']]
# df_t = df_t.drop_duplicates()
# df_t = df_t.dropna(subset=['msa'])

#df_t.to_csv('origin/msa_name.csv')
#meta_df.loc[meta_df.msa == '31', :]

In [33]:
meta_df = meta_df[['fipscounty', 'msa', 'msaname']]

In [34]:
msa2fips = {}
for name, gr in meta_df.groupby('msa'):
    tmp = list(set(gr['fipscounty'].to_list()))
    msa2fips[name] = []
    for df in dfs:
        for fips in tmp:
            if fips in df.index:
                msa2fips[name].append(fips)
for k, v in msa2fips.items():
    msa2fips[k] = list(set(v))

In [45]:
sum([len(v) for k, v in msa2fips.items()])

854

In [46]:
len(msa2fips)

324

In [ ]:
num_msa = len(msa2fips)
list_msa = list(msa2fips.keys())
for i, df in tqdm(enumerate(dfs)):
    dff = pd.DataFrame(np.zeros((num_msa, num_msa)), 
                       columns=list_msa, 
                       index=list_msa)
    for transmitter in list_msa:
        for receiver in list_msa:
            t_indices = msa2fips[transmitter]
            r_indices = msa2fips[receiver]

            np_node = df.loc[t_indices, r_indices].to_numpy()
            if transmitter == receiver:
                dff.loc[transmitter, receiver] = np.sum(np_node) - np.sum(np.diagonal(np_node))
            else:
                dff.loc[transmitter, receiver] = np.sum(np_node)
    dff.to_csv('./msa/IRS_Migration_Matrix_%d.csv' % YEARS[i])

2it [03:27, 103.83s/it]

In [38]:
2012-1990+1

23

In [47]:
num_msa = len(msa2fips)
list_msa = list(msa2fips.keys())
for i, df in tqdm(enumerate(dfs)):
    if i < 23:
        continue
    dff = pd.DataFrame(np.zeros((num_msa, num_msa)), 
                       columns=list_msa, 
                       index=list_msa)
    for transmitter in list_msa:
        for receiver in list_msa:
            t_indices = msa2fips[transmitter]
            r_indices = msa2fips[receiver]

            np_node = df.loc[df.index.intersection(t_indices), 
                             df.index.intersection(r_indices)].to_numpy()
            if transmitter == receiver:
                dff.loc[transmitter, receiver] = np.sum(np_node) - np.sum(np.diagonal(np_node))
            else:
                dff.loc[transmitter, receiver] = np.sum(np_node)
    dff.to_csv('./msa/IRS_Migration_Matrix_%d.csv' % YEARS[i])

29it [14:28, 29.94s/it] 


In [ ]:
df.loc[df.index.intersection(['01000']), df.index.intersection(['01000'])]

In [ ]:
df.index.intersection(['01000'])